<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline

import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Multidimensional Problems

We now will consider multidimensional hyperbolic PDEs of the form
$$
    q_t + f(q)_x + g(q)_y = 0
$$
in conservation form and in the quasilinear form
$$
    q_t + A(q, x, y, t) q_x + B(q, x, y, t) q_y = 0.
$$
Analogously we have
$$
    q_t + f(q)_x + g(q)_y + h(q)_z = 0
$$
in three dimensions.

## Derivation of Conservation Laws

Similar to the 1 dimensional case we consider some conserved quantity inside of a spatial domain $\Omega$ and consider the time change in the quantity inside of the domain.  This is commonly written as
$$
    \frac{\text{d}}{\text{d} t} \int \int_\Omega q(x, y, t) dx dy = \text{net flux across } \partial \Omega
$$
where $\partial \Omega$ is the boundary of $\Omega$.

If $f(q)$ is the flux across the boundary in the x-direction with units of per unit length in $t$ and per unit time then we will consider an interval (x_0, y_0) \times (x_0, y_0 + \Delta y) over time $\Delta t$ leading to
$$
    \Delta t \Delta y f(q(x_0, y_0, t))
$$
for $\Delta y$ and $\Delta t$ sufficiently small.  The same follows for the y-direction.

Generalizing these ideas let
$$
    \vec{~f}(q) = (f(q), g(q))
$$
be the flux vector and
$$
    \vec{~n}(s) = (n^x(s), n^y(s))
$$
be the outward-pointing unit normal vector to $\partial \Omega$ as some point $\vec{~x}(s) = (x(s), y(s))$ on $\partial \Omega$ ($s$ is usually taken to be the arc-length parameterization on $\partial \Omega$).  Then the flux in the direction of $\vec{~n}(s)$ is
$$
    \vec{~n}(s) \cdot \vec{~f}(q(x(s), y(s), t))
$$
and therefore we have
$$\begin{aligned}
    \frac{\text{d}}{\text{d} t} \int \int_\Omega q(x, y, t) dx dy &= \text{net flux across } \partial \Omega \\
    &= - \int_{\partial \Omega} \vec{~n}(s) \cdot \vec{~f}(q) ds \\
    &= - \int \int_{\Omega} \vec{~\nabla} \cdot \vec{~f}(q) dx dy
\end{aligned}$$
where we have assumed that $q$ is smooth in the last step and used the divergence theorem.  This of course then leads to
$$
    \int \int_\Omega [q_t + \vec{~\nabla} \cdot \vec{~f}(q)] dx dy = 0
$$
and therefore the strong form of the conservation law.

Simplifying this a bit let's consider a more specific region $\Omega = (x_{i-1}, x_{i}) \times (y_{j-1}, y_j)$. Now it's much more clear that we need to integrate the flux $f(q)$ over the faces aligned with $x_{i-1}$ and $x_i$ and the flux $g(q)$ over the faces with $y_{j-1}$ and $y_j$ leading to
$$
    \frac{\text{d}}{\text{d} t} \int^{y_j}_{y_{j-1}} \int^{x_i}_{x_{i-1}} q(x, y, t) dx dy = \int^{y_j}_{y_{j-1}} (f(q(x_{i-1}, y, t) - f(q(x_i, y, t)) dy + \int^{x_i}_{x_{i-1}} (g(q(x, y_{j-1}, t) - g(q(x, y_j, t)) dx.
$$
Again for smooth situations this can be rewritten using the FTC as the strong form from before.

## Advection

For advection with possibly variable speeds we have the fluxes
$$
    f = u(x, y, t) q(x, y, t) \quad \text{and} \quad g = v(x, y, t) q(x, y, t)
$$
so that we have
$$
    q_t + (uq)_x + (vq)_y = 0.
$$
This of course simplifies if $u$ and $v$ are independent of space and time so that we know the solution is
$$
    q(x, y, t) = q(x - u t, y - v t, 0).
$$

### Acoustics

As an example of a system of linear system of equations let's again consider the acoustics system.  We again have perturbations from some rest state defined by $q_0$ such that
$$
    q_t + f'(q_0) q_x + g'(q_0) q_y = 0
$$
where now $q$ is the perturbation to $q_0$.  The Jacobians with $u_0 = v_0 = 0$ then are
$$
    f'(q_0) = \begin{bmatrix}
        0 & 1 & 0 \\
        P'(\rho_0) & 0 & 0 \\
        0 & 0 & 0
    \end{bmatrix} \quad g'(q_0) = \begin{bmatrix}
        0 & 0 & 1 \\
        0 & 0 & 0 \\
        P'(\rho_0) & 0 & 0 \\
    \end{bmatrix}.
$$
This system can then be written as
$$
    q_t + A q_x + B q_y = 0
$$
where
$$
    q = \begin{bmatrix} p \\ u \\ v \end{bmatrix} \quad
    A = \begin{bmatrix}
        0 & K_0 & 0 \\
        \frac{1}{\rho_0} & 0 & 0 \\
        0 & 0 & 0
    \end{bmatrix} \quad B = \begin{bmatrix}
        0 & 0 & K_0 \\
        0 & 0 & 0 \\
        \frac{1}{\rho_0} & 0 & 0 \\
    \end{bmatrix}.
$$

## Hyperbolicity

In multiple spatial dimensions it is not entirely clear how to extend our notion of hyperbolicity to one dimension to multiple.  It turns out we need the condition that the matrices $A$ and $B$ are both diagonalizable with real eigenvalues but also that any combination of these matrices have the same condition.

Why might this latter condition be needed?

The critical addition here is that we want wave-like behavior not only in the coordinate aligned directions but in any direction.  The other way to look at this is that an affine transformation of the coordinate axes should not change the nature of the problem (by a rotation for instance).

This can be formalized by saying that for any $\vec{~n}$ that waves should propagate as
$$
    q(x, y, t) = \widetilde{q}(\vec{~n} \cdot \vec{~x} - s t).
$$
This also requires then that
$$
    \widetilde{A} \widetilde{q}(\vec{~n} \cdot \vec{~x} - s t) = s \widetilde{q}(\vec{~n} \cdot \vec{~x} - s t)
$$
where
$$
    \widetilde{A} = \vec{~n} \cdot \vec{~A} = n^x A + n^y B.
$$

This then leads to the following definition for hyperbolicity:

The constant-coefficient system 
$$
    q_t + A q_x + B q_y = 0
$$ 
is (strongly) **hyperbolic** if, for every choice of $\vec{~n}$, the matrix $\vec{~A} = \vec{~n} \cdot \vec{~A}$ is diagonalizable with real eigenvalues.  The quasilinear system 
$$
    q_t + f'(q) q_x + g'(q) q_y = 0
$$
is **hyperbolic** in some region of state space if the Jacobian matrix
$$
    \widetilde{f}'(q) = \vec{~n} \cdot \vec{~f}(q) = n^x f'(q) + n^y g'(q)
$$
is diagonalizable with real eigenvalues for every $\vec{~n}$, for all $q$ in the region.

Let's now check to see that this makes sense.  If we know that the matrices are diagonalizable then we know that for a given coordinate system that
$$
    A = R^x \Lambda^x (R^x)^{-1} \quad B = R^y \Lambda^y (R^y)^{-1}
$$
meaning that we cannot simply transform the equations into the characteristic variables.  If the eigensystem has the same eigenvectors then we could do this leading to
$$
    w_t + \Lambda^x w_x + \Lambda^y w_y = 0
$$
leading to something similar from the 1-dimensional case.  Unfortunately this is often not the case.

## Shallow Water Equations

In two space dimensions the shallow water equations are
$$\begin{aligned}
    &h_t + (hu)_x + (hv)_y = 0 \\
    &(hu)_t + \left( hu^2 + \frac{1}{2} g h^2 \right)_x + (huv)_y = 0 \\
    &(hv)_t + (huv)_x + \left( hv^2 + \frac{1}{2} g h^2 \right)_y = 0
\end{aligned}$$
leading to the Jacobians
$$
    f'(q) = \begin{bmatrix}
        0 & 1 & 0 \\
        -u^2 + gh & 2u & 0 \\
        -uv & v & u
    \end{bmatrix} \quad g'(q) = \begin{bmatrix}
        0 & 0 & 1 \\
        -uv & v & u \\
        -v^2 + gh & 0 & 2v
    \end{bmatrix}.
$$

The eigenproblem then leads to
$$
    \Lambda^x = \begin{bmatrix}
        u - \sqrt{gh} & 0 & 0 \\
        0 & u & 0 \\
        0 & 0 & u + \sqrt{gh}
    \end{bmatrix} \quad \Lambda^y = \begin{bmatrix}
        v - \sqrt{gh} & 0 & 0 \\
        0 & v & 0 \\
        0 & 0 & v + \sqrt{gh}
    \end{bmatrix}
$$
and
$$
    R^x = \begin{bmatrix}
        1 & 0 & 1 \\
        u - \sqrt{gh} & 0 & u + \sqrt{gh} \\
        v & 1 & v
    \end{bmatrix} \quad R^y = \begin{bmatrix}
        1 & 0 & 1 \\
        u & -1 & u \\
        v - \sqrt{gh} & 0 & v + \sqrt{gh}
    \end{bmatrix}
$$

# Numerical Methods for Multiple Dimensions

We now aim to approximate the cell average
$$
    Q^n_{ij} \approx \frac{1}{\Delta x \Delta y} \int^{y_{j+1/2}}_{y_{j-1/2}} \int^{x_{i+1/2}}_{x_{i-1/2}} q(x, y, t_n) dx dy
$$

## Finite Difference Methods

It can be useful at this juncture to review some of the multi-dimensional methods for finite differences.  Here instead of the goal of approximating cell averages we are approximating the point-wise approximations with
$$
    Q^n_{ij} \approx q(x_i, y_j, t_n).
$$

### Taylor-Series Expansions

Taylor-series usually is the start of any analysis for finite difference methods and this is no different.  Consider the linear hyperbolic system
$$
    q_t + A q_x + B q_y = 0.
$$
Assuming appropriate smoothness we can derive relationships using the equation itself.  The general expression can be rewritten as a recursive operator as
$$
    \partial_t^{(j)} q = [ - (A \partial_x + B \partial_y )]^{j} q
$$
so that for instance we can find the second time derivative as
$$
    q_{tt} = A^2 q_{xx} + A B q_{yx} + B A q_{xy} + B^2 q_{yy}.
$$
Note that in general we can permute the matrices however derivative order can due our assumption of smoothness.

$$
    q_{tt} = A^2 q_{xx} + A B q_{yx} + B A q_{xy} + B^2 q_{yy}
$$
From here we can build the time derivative as
$$\begin{aligned}
    q(x_i, y_j, t_n + \Delta t) &= q + \Delta t q_t + \frac{\Delta t^2}{2} q_{tt} + \cdots \\
    &= q - \Delta t (A q_x + B q_y) + \frac{\Delta t^2}{2} \left( A^2 q_{xx} + A B q_{yx} + B A q_{xy} + B^2 q_{yy} \right) + \cdots
\end{aligned}$$
This generally holds unless $A$ and $B$ also vary with $x$ and $y$, which then leads to the expression
$$
    q_{tt} = A (A q_{x})_x + A (B q_{y})_x + B (A q_{x})_y + B (B q_{y})_y
$$
instead.

### The Lax-Wendroff Method

We can now take this expression and using centered finite differences discretize the derivatives.  The trickier now though is that we have cross-derivative terms.  For the second derivatives we have the common centered difference of
$$
    q_{yy} = \frac{1}{\Delta y^2} (Q^n_{i,j-1} - 2 Q^n_{ij} + Q^n_{i,j+1}).
$$
For the cross-derivatives we then have
$$
    q_{xy} = q_{yx} \approx \frac{1}{4 \Delta x \Delta y} [(Q^n_{i+1,j+1} - Q^n_{i-1, j+1}) - (Q^n_{i+1,j-1} - Q^n_{i-1, j-1})]
$$
that are centered derivatives applied in each directions.  This leads to the final approximation of
$$\begin{aligned}
    Q^{n+1}_{ij} &= Q^n_{ij} - \frac{\Delta t}{2 \Delta x} A (Q^n_{i+1,j} - Q^n_{i-1, j}) - \frac{\Delta t}{2 \Delta y} B (Q^n_{i, j+1} - Q^n_{i, j-1}) \\
    &\quad+ \frac{\Delta t^2}{2 \Delta x^2} A^2 (Q^n_{i+1,j} - 2 Q^n_{ij} + Q^n_{i-1, j}) + \frac{\Delta t^2}{2 \Delta y^2} B^2 (Q^n_{i,j+1} - 2 Q^n_{ij} + Q^n_{i,j-1}) \\
    &\quad+ \frac{\Delta t^2}{8 \Delta x \Delta y} (AB + BA) [(Q^n_{i+1,j+1} - Q^n_{i-1, j+1}) - (Q^n_{i+1,j-1} - Q^n_{i-1, j-1})]
\end{aligned}$$

## Finite Volume Methods

As we used the Lax-Wendroff method before to create a second-order accurate finite volume method we will do the same here.  Instead of following directly though we will immediately introduce our Riemann problem machinery and limiting to avoid the dispersion issues we usually see Lax-Wendroff method.  Before moving on however we will look at some other alternatives before looking delving into this.

### Fully Discrete Methods

Fully-discrete methods, such as flux-differencing forms of Lax-Wendroff, define a flux across each grid cell edge and use this to compute the method updates.  Note that we still need to provide limited fluxes in order to avoid oscillatory solutions.

#### Flux-Differencing Approaches

Let's return to the flux versions of our conservation law (the nonlinear one) and consider a rectangular cell 
$$
    \mathcal{C}_{ij} = [x_{i-1/2}, x_{i+1/2}] \times [y_{j-1/2}, y_{j+1/2}] = [x_{i-1/2}, x_{i-1/2} + \Delta x] \times [y_{j-1/2}, y_{j-1/2} + \Delta y].
$$
Integrating around the edges of this cell we have
$$\begin{aligned}
\frac{\text{d}}{\text{d}t} \int\int_{\mathcal{C}_{ij}} q(x, y, t) dx dy &= \int^{y_{j+1/2}}_{y_{j-1/2}} f(q(x_{i+1/2}, y, t)) dy - \int^{y_{j+1/2}}_{y_{j-1/2}} f(q(x_{i-1/2}, y, t)) dy \\
&\quad \int^{x_{i+1/2}}_{x_{i-1/2}} g(q(x, y_{j+1/2}, t)) dx - \int^{x_{i+1/2}}_{x_{i-1/2}} g(q(x, y_{j-1/2}, t)) dx.
\end{aligned}$$

Integrating in time and dividing by the cell area we then are lead to
$$
    Q^{n+1}_{ij} = Q^n_{ij} - \frac{\Delta t}{\Delta x} [F^n_{i-1/2,j} - F^n_{i+1/2,j}] - \frac{\Delta t}{\Delta y} [G^n_{i,j-1/2} - G^n_{i,j+1/2}]
$$
where
$$\begin{aligned}
    F^n_{i-1/2,j} &\approx \frac{1}{\Delta t \Delta y} \int^{t_{n+1}}_{t_n} \int^{y_{j+1/2}}_{y_{j-1/2}} f(q(x_{i-1/2}, y ,t) dy dt\\
    G^n_{i,j-1/2} &\approx \frac{1}{\Delta t \Delta x} \int^{t_{n+1}}_{t_n} \int^{x_{i+1/2}}_{x_{i-1/2}} g(q(x, y_{j-1/2} ,t) dx dt.
\end{aligned}$$

For the simpler case of a linear system we can also rewrite the Taylor expansion from before as
$$\begin{aligned}
    q(x_i, y_j, t_n + \Delta t) &= q - \Delta t \left(Aq - \frac{\Delta t}{2} A^2 q_x - \frac{\Delta t}{2} A B q_y \right)_x \\
    &\quad - \Delta t \left(Bq - \frac{\Delta t}{2} B^2 q_y - \frac{\Delta t}{2} BA q_x \right)_y + \cdots
\end{aligned}
$$
suggesting the following forms of the numerical flux functions
$$\begin{aligned}
    F^n_{i-1/2,j} &\approx A q(x_{i-1/2}, y_j, t_n) - \frac{\Delta t}{2} A^2 q_x(x_{i-1/2}, y_j, t_n) - \frac{\Delta t}{2} A B q_y (x_{i-1/2}, y_j, t_n)\\
    G^n_{i,j-1/2} &\approx B q(x_i, y_{j-1/2}, t_n) - \frac{\Delta t}{2} B^2 q_y(x_i, y_{j-1/2}, t_n) - \frac{\Delta t}{2} BA q_y (x_i, y_{j-1/2}, t_n)
\end{aligned}$$
that agree with the integral definition to $\mathcal{O}(\Delta t^2)$.

We can also write the flux-differencing form of the Lax-Wendroff method for a constant coefficient system as the following:
$$\begin{aligned}
    F_{i-1/2, j} &= \frac{1}{2} A (Q_{i-1,j} + Q_{ij}) - \frac{\Delta t}{2 \Delta x} A^2 (Q_{ij} - Q_{i-1,j}) \\
    &\quad -\frac{\Delta t}{8 \Delta y} AB [(Q_{i,j+1} - Q_{ij}) + Q_{i-1,j+1} - Q_{i-1, j} \\
    &\quad \quad + (Q_{ij} - Q_{i,j-1}) + (Q_{i-1,j} - Q_{i-1, j-1})] \\
    G_{i, j-1/2} &= \frac{1}{2} B (Q_{i,j-1} + Q_{ij}) - \frac{\Delta t}{2 \Delta y} B^2 (Q_{ij} - Q_{i,j-1}) \\
    &\quad -\frac{\Delta t}{8 \Delta x} BA [(Q_{i+1,j} - Q_{ij}) + Q_{i+1,j-1} - Q_{i, j-1} \\
    &\quad \quad + (Q_{ij} - Q_{i-1,j}) + (Q_{i,j-1} - Q_{i-1, j-1})] \\
\end{aligned}$$

#### Godunov's Method

The extension to Godunov's method simply does what we have been doing the entire semester, evaluate the Riemann problem, find the value of $Q$ at the grid cell edge, and evaluate the flux function there
$$
    F_{i-1/2, j} = f(\widehat{Q}_{i-1/2, j}) \quad \quad G_{i, j-1/2} = g(\widehat{Q}_{i, j-1/2}).
$$
This seems straight forward except that now we have a multidimensional Riemann problem to solve.  Instead we often only solve in each direction.  In other words we find $\widehat{Q}_{i-1/2, j}$ from $q_t + f(q)_x = 0$ and $\widehat{Q}_{i, j-1/2}$ from $q_t + g(q)_y = 0$.

If we again consider a linear problem we know that we can as before define
$$
    A^\pm = R^x (\Lambda^x)^\pm (R^x)^{-1} \quad \quad B^\pm = R^y (\Lambda^y)^\pm (R^y)^{-1}
$$
that allows us to define our usual fluxes
$$
    F_{i-1/2, j} = A^+ Q_{i-1,j} + A^- Q_{ij} \quad \quad G_{i, j-1/2} = B^+ Q_{i,j-1} + B^- Q_{ij}.
$$
This is of course only a first order method.

We can also extend the to not only general conservation laws but also add limited corrections easily in our usual notation:
$$\begin{aligned}
    Q^{n+1}_{ij} &= Q^n_{ij} - \frac{\Delta t}{\Delta x} \left (\mathcal{A}^+\Delta Q_{i-1/2,j} + \mathcal{A}^-\Delta Q_{i+1/2, j} \right) \\
    &\quad- \frac{\Delta t}{\Delta y} \left (\mathcal{B}^+\Delta Q_{i,j-1/2} + \mathcal{B}^-\Delta Q_{i, j+1/2} \right) \\
    &\quad -\frac{\Delta t}{\Delta x} \left(\widetilde{F}_{i+1/2,j} - \widetilde{F}_{i-1/2,j} \right ) - \frac{\Delta t}{\Delta y} \left(\widetilde{G}_{i,j+1/2} - \widetilde{G}_{i,j-1/2} \right )
\end{aligned}$$

### Semi-Discrete Approaches

We can also consider semi-discrete, or method-of-lines, discretization where we are instead considering the approximation
$$
    Q_{ij}(t) = \int \int_{\mathcal{C}_{ij}} q(x, y, t) dx dy
$$
leading to 
$$\begin{aligned}
    F_{i-1/2, j}(Q(t)) &\approx \frac{1}{\Delta y} \int^{y_{j+1/2}}_{y_{j-1/2}} f(q(x_{i-1/2},y,t)) dy \\
    G_{i, j-1/2}(Q(t)) &\approx \frac{1}{\Delta x} \int^{x_{i+1/2}}_{x_{i-1/2}} g(q(x,y_{j-1/2},t)) dx.
\end{aligned}$$

This can also be rewritten in a more familiar, semi-discrete form as
$$\begin{aligned}
    \frac{\text{d}}{\text{d} t} Q_{ij}(t) = &-\frac{1}{\Delta x} \left [ F_{i+1/2,j} (Q(t)) - F_{i-1/2, j}(Q(t)) \right] \\
    &-\frac{1}{\Delta y} \left [ G_{i,j+1/2} (Q(t)) - G_{i, j-1/2}(Q(t)) \right] \\
\end{aligned}$$

### Dimensional Splitting

The final alternative to what we eventually discuss is simply to use dimensional splitting.  The primary idea is to split the different dimensions into sweeps and combine their changes.  The basic algorithm for this is to split
$$
    q_t + A q_x + B q_y = 0
$$
into subproblems
$$\begin{aligned}
    &x-\text{sweeps:} \quad q_t + A q_x = 0 \\
    &y-\text{sweeps:} \quad q_t + B q_y = 0.
\end{aligned}$$

This amount to evolving the function in two steps where
$$\begin{aligned}
    Q^*_{ij} &= Q^n_{ij} - \frac{\Delta t}{\Delta x} (F^n_{i+1/2, j} - F^n_{i-1/2,j}) \\
    Q^{n+1}_{ij} &= Q^*_{ij} - \frac{\Delta t}{\Delta y} (G^*_{i+1/2, j} - G^*_{i-1/2,j}).
\end{aligned}$$
The primary question those is what error we impose by picking a direction to use first.

This splitting is know as Godunov splitting.  We know that we can formally achieve second order convergence with Strang splitting but is this going to work?

## Multidimensional Finite Volume Advection Methods

In order to illustrate the ideas of fully multidimensional methods that do not depend on our previous approaches we will focus on the constant coefficient advection equation
$$
    q_t + u q_x + v q_y = 0
$$
as it can provide the basic generalizations needed for our more general discussion.

A useful analytical tool (as usual) will be the Taylor series expansion of the solution
$$\begin{aligned}
    q(x, y ,t_{n+1}) &= q + \Delta t q_t + \frac{\Delta t^2}{2} q_{tt} + \cdots \\
    &= q - u \Delta t q_x - v \Delta t q_y + \frac{\Delta t^2}{2} [u^2 q_{xx} + vu q_{xy} + uv q_{yx} + v^2 q_{yy}] + \cdots
\end{aligned}$$
despite the fact we know the true solution is
$$
    q(x,y,t) = q(x-ut, y-vt, 0).
$$

### Donor-Cell Upwind Method

The first and most straight-forward FV method for advection is first-order upwind and can be written as
$$
    Q^{n+1}_{ij} = Q^n_{ij} - \frac{\Delta t}{\Delta x} [u^+(Q^n_{ij} - Q^n_{i-1,j}) + u^- (Q^n_{i+1,j} - Q^n_{ij})] - \frac{\Delta t}{\Delta y} [v^+(Q^n_{i,j-1} - Q^n_{ij}) + v^- (Q^n_{i,j+1} - Q^n_{ij})].
$$
This is method has the fluctuations specified as
$$
    \mathcal{A}^\pm \Delta Q_{i-1/2,j} = u^\pm(Q_{ij} - Q_{i-1, j}) \quad \mathcal{B}^\pm \Delta Q_{i,j-1/2} = v^\pm(Q_{ij} - Q_{i, j-1}) 
$$
that agrees with Godunov's method as described before.

As we might expect this method can be problematic given that we do not expect that waves that should be moving at some angle to the grid will yield a true CFL stable solution.  In fact this method is only stable when
$$
    \left | \frac{u\Delta t}{\Delta x}\right | + \left | \frac{v\Delta t}{\Delta y}\right | \leq 1.
$$
Note that this is not simply the maximum by additive greatly reducing the time step we can take depending on the value of $u$ and $v$.

### Corner-Transport Upwind Method

Instead of the Donor-Cell method we can fall back to the REA approach and extend it to two dimensions:
1. Define a piecewise constant function $\widetilde{q}^n(x, y, t_n)$ with constant value $Q^n_{ij}$ defined over $\mathcal{C}_{ij}$.
1. Evolve the full advection equation exactly over $\Delta t$.
1. Average the resulting solution $\widetilde{q}^n(x,y,t_{n+1})$ back onto the grid.

We exactly know the approximation that arises from this approximation evolution as
$$
    \widetilde{q}^n(x, y, t_{n+1}) = \widetilde{q}^n(x - u \Delta t, y - v \Delta t, t_n).
$$
We also therefore know that the piecewise constant representation of the solution to the advection problem is simply shifted by $(u \Delta t, v \Delta t)$ so that we have
$$\begin{aligned}
    Q^{n+1}_{ij} &= \frac{1}{\Delta x \Delta y} \int^{x_{i+1/2}}_{x_{i-1/2}} \int^{y_{j+1/2}}_{y_{j-1/2}} \widetilde{q}^n(x - u \Delta t, y - v \Delta t, t_n) dx dy \\
    &= \frac{1}{\Delta x \Delta y} \int^{x_{i+1/2} - u \Delta t}_{x_{i-1/2} - u \Delta t} \int^{y_{j+1/2} - v \Delta t}_{y_{j-1/2} - v \Delta t} \widetilde{q}^n(x, y, t_n) dx dy
\end{aligned}$$

This leads to the final update formula
$$\begin{aligned}
    Q^{n+1}_{ij} &= \frac{1}{\Delta x}{\Delta y} \left[ (\Delta x - u \Delta t)(\Delta y - v \Delta t) Q^n_{ij} + (\Delta x - u \Delta t)( v \Delta t) Q^n_{i,j-1} \right . \\
    & \left . \quad \quad + (\Delta y - v \Delta t)(u \Delta t) Q^n_{i-1,j} + (u \Delta t) (v \Delta t) Q^n_{i-1, j-1}\right].
\end{aligned}$$

### Wave-Propagation Implementation of Corner-Transport

How does this work into our wave-propagation formulation beyond first order?  We can actually define correction fluxes as the following
$$\begin{aligned}
    &\widetilde{F}_{i-1/2, j} = -\frac{\Delta t}{2 \Delta y} uv(Q_{i-1,j} - Q_{i-1,j-1}) \\
    &\widetilde{F}_{i+1/2, j} = -\frac{\Delta t}{2 \Delta y} uv(Q_{i,j} - Q_{i,j-1}) \\
    &\widetilde{F}_{i, j-1/2} = -\frac{\Delta t}{2 \Delta y} uv(Q_{i,j-1} - Q_{i-1,j-1}) \\
    &\widetilde{G}_{i, j+1/2} = -\frac{\Delta t}{2 \Delta y} uv(Q_{i,j} - Q_{i-1,j})
\end{aligned}$$